# 5pDEER DEMO
 The aim of this script is to set out the basic design/implemntaion for running incoherent pump pulses on the C Floor AWG.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autoDeer.hardware.keysight_awg as ksawg
import autoDeer.hardware.pulses as pulses
import autoDeer.hardware.xepr_experiments as exp
from autoDeer.ResPro import resonatorProfile
from autoDeer.FieldSweep import FieldSweep
from autoDeer.hardware import xepr_api as api
import autoDeer.Param_Optimization as po
import time


## Setup

### Connecting to the AWG
The Sequencing mode on both channels should be "STSC"

In [2]:
awg = ksawg.Interface()
awg.open_con('129.132.218.87')
awg._inst_init()
print("Checking Sequencing mode:")
awg.getFunctionMode(3)
sampling_freq = 12e9
sampling_period = 1/sampling_freq
grad = 64


Socket Created
Identification String: b'Agilent Technologies,M8190A,DE52200268,5.0.14.0-2\n'
Checking Sequencing mode:


## Connecting to Bruker

In [ ]:
xepr=api() # This can only be run once per kernel
xepr.find_Xepr()
xepr.find_cur_exp()
xepr.find_hidden()

## Resonator Profile

Go to the central frequency of the dip.
Find the rough field maximum and tune +-\<x\> channels for 16ns pulses

In [ ]:
exp.run_general(xepr,
    ["/PulseSpel/HUKA_DEER_AWG"],
    ["Field Sweep +<x>/-<x> pg=200ns","Field sweep +<x>/-<x>"],
    {"PhaseCycle":False},
    {"p0":16,"p1":16,"h":20,"n":2})

fs_data = xepr.acquire_scan()
fs = FieldSweep()
fs.import_from_dataclass(fs_data)
fs.find_max()

xepr.set_field(fs.field_max)
Bc = xepr.get_field()
fc = xepr.get_counterfreq()

fs.plot()

Confirm that two consecutative nutation experiments produce the same result. This confirms that everythinhg is set up correctly.

In [ ]:
exp.run_general(xepr,
    ["/PulseSpel/HUKA_DEER_AWG"],
    ["AWG seq inv (p2)","AWG +-<x> obs"],
    {"PhaseCycle":True},
    {"p0":16,"p1":16,"h":20,"n":1},False)
xepr.run_exp()
run1 = xepr.acquire_scan()

xepr.run_exp()
run2 = xepr.acquire_scan()
plt.plot(run1.time,np.real(run1.data),
        run2.time,np.real(run2.data))
plt.legend()

Now run the resonator profile script

In [ ]:
t,nutation_data = exp.get_nutations(xepr,
                                    nu=[33, 34],
                                    field=[fc,Bc],
                                    step=20e-3)

Finding and fitting the resonantor profile

In [ ]:
rp = resonatorProfile()
res_prof = rp.calc_res_prof(nutation_data,dt=t[1]-t[0])
rp.autofit([33,35])
rp.calculate_shape(res_prof,None)
rp.calc_IF_prof(2,rp.fc,type='fit')
rp.res_prof_plot([33,35])


Now go to the calculated fc, and do another field sweep. This is the new central frequency

In [ ]:
exp.run_general(xepr,
    ["/PulseSpel/HUKA_DEER_AWG"],
    ["Field Sweep +<x>/-<x> pg=200ns","Field sweep +<x>/-<x>"],
    {"PhaseCycle":False},
    {"p0":16,"p1":16,"h":20,"n":2})

fs_data = xepr.acquire_scan()
fs = FieldSweep()
fs.import_from_dataclass(fs_data)
fs.find_max()
xepr.set_field(fs.field_max)
Bc = xepr.get_field()
fc = xepr.get_counterfreq()
fs.plot()

## Relaxation Experiments

These need to be done on the normal channels. So tune the main channel

In [ ]:
awg.Abort() # Make sure AWG has stopped
po.main_run(xepr,[16,32],d0,)

## DEER Experiments

### Building the Waveforms

In [9]:
tpump = 100
nu = [1.6+0.07,1.6+0.07+0.15]
[p_real,p_imag] = pulses.HSorder1(12,
                                tpump,
                                nu,
                                HSbeta=10,
                                HSorder=[1,6],
                                phase=0,
                                scale=1)
wave1 = ksawg.Waveform(awg,p_real)
wave1.pulse_blanking()
wave1.enforce_gradualrity()
wave1.set_amp('max')
wave1._awg_check()
# wave1.plot().show()
wave2 = ksawg.Waveform(awg,p_imag)
wave2.pulse_blanking()
wave2.enforce_gradualrity()
wave2.set_amp('max')
wave2._awg_check()
# wave2.plot().show()

[p_real,p_imag] = pulses.HSorder1(12,
                                tpump,
                                np.flip(nu),
                                HSbeta=10,
                                HSorder=np.flip([1,6]),
                                phase=0,
                                scale=1)
wave3 = ksawg.Waveform(awg,p_real)
wave3.pulse_blanking()
wave3.enforce_gradualrity()
wave3.set_amp('max')
wave3._awg_check()
# wave3.plot().show()
wave4 = ksawg.Waveform(awg,p_imag)
wave4.pulse_blanking()
wave4.enforce_gradualrity()
wave4.set_amp('max')
wave4._awg_check()
# wave4.plot().show()


True

#### Sending Waveform to AWG

In [ ]:
awg.Abort(3)
awg.deleteAllWaveforms(3)
wave1.define_new_waveform(1,ch=3)
wave2.define_new_waveform(2,ch=3)
wave1.send_waveform(1)
wave2.send_waveform(2)

In [8]:
# Old Code
awg.Abort(3)
awg.deleteAllWaveforms(3)
awg.defineWaveform(1,3008,0,ch=3)
awg.defineWaveform(2,3008,0,ch=3)
awg.setWaveform(wave1,1,ch=1)
awg.setWaveform(wave2,2,ch=1)
awg.setWaveform(wave3,1,ch=2)
awg.setWaveform(wave4,2,ch=2)

### Building the Sequence Table

In [6]:
seq1 = ksawg.Sequence(awg)
seq1.encode(
    {"data_cmd":0,
"init_marker_seq":1,
"marker_enable":1,
"seg_adv_mode":3,
"seq_loop_cnt":1,
"seg_loop_cnt":1,
"seg_id":1,
"seg_start_offset":0,
"seg_end_offset":3007}
 )
seq2 = ksawg.Sequence(awg)
seq2.encode(
    {"data_cmd":0,
"init_marker_seq":0,
"end_marker_seq":1,
"end_marker_sce":1,
"marker_enable":1,
"seg_adv_mode":3,
"seq_loop_cnt":1,
"seg_loop_cnt":1,
"seg_id":2,
"seg_start_offset":0,
"seg_end_offset":3007}
 )


285409280
1627586560


'1627586560,1,1,2,0,3007'

In [7]:
seq_table = ksawg.SequenceTable(awg)
seq_table.table = [seq1,seq2]
seq_table.write_to_str()

'285409280,1,1,1,0,3007,1627586560,1,1,2,0,3007'

In [10]:
awg.resetSequenceTable()
awg.setSequenceTable(seq_table,0,3)